In [ ]:
import pickle
import pandas as pd
import numpy as np
import matplotlib
import geopandas
import os.path
import rpy2
import scorecardpy as sc

In [ ]:
f = os.path.expanduser("~/Documents/Vodacom/data/df_raw_final.pkl")

#reading from datafile
df = pd.read_pickle(f)

#datafile = df

In [ ]:
#df.geometry

In [ ]:
#df.D_LEARNERS_KM2.mean()
#df.D_MIGR_IN_KM2.max()
#testing statistical inferences on panda

In [ ]:
dfclean = df.dropna(subset = ['target_label'])

#new data set after dropping all values with NaN

In [ ]:
dfclean = dfclean.replace(to_replace = "yes", value = 1)
dfclean = dfclean.replace(to_replace = "no", value = 0)
dfclean = dfclean.replace(to_replace = "maybe", value = 2)

#replace all strings "yes", "no", "maybe" with integers 0, 1, 2

In [ ]:
dfclean = dfclean[dfclean.target_label != 2]

#remove all rows with target_label = 2 ("maybe")

In [ ]:
dfclean_nogeo = dfclean.drop(columns = "geometry")

#remove column geometry due to string error with conversion


dfclean_nogeo.human_label
# dfclean_nogeo.info()
# dfclean_nogeo.describe(include = 'all')

In [ ]:
# feat_sum = pd.DataFrame()
# feat_sum = pd.DataFrame(dfclean_nogeo.dtypes)
# i = 1

# for feature in dfclean_nogeo:
#         feat_sum.loc[i-1:3] = dfclean_nogeo[feature].nunique()
#         feat_sum.loc[i-1:4] = dfclean_nogeo[feature].isna().sum()
        
#         i = i +1
        
# feat_sum

In [ ]:
dfclean_nogeo.human_label.nunique()

In [ ]:
if (dfclean_nogeo['EA_CODE'].dtype == 'object'):
    
    #dfclean_nogeo['EA_CODE'].astype('category')
    
elif (dfclean_nogeo['EA_CODE'].dtype == 'float64' or dfclean_nogeo['EA_CODE'].dtype =='int64'):
    
    my_list = dfclean_nogeo['EA_CODE'].values
    new_list = []
    for item in my_list:
        new_list.append(len(str(item)))
    
    len(set(new_list)) == 1 
    #dfclean_nogeo['EA_CODE'].astype('category')

else     dfclean_nogeo['EA_CODE'].astype('numerical')

In [ ]:
my_list = dfclean_nogeo['EA_CODE'].values

new_list = []
for item in my_list:
    new_list.append(len(str(item)))
    
len(set(new_list)) == 1 


In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R -i dfclean_nogeo -o cutp

library(smbinning)

sbin = smbinning(dfclean_nogeo, 'target_label', 'D_MIGR_IN_KM2')

cutp = sbin$bands 

In [ ]:
cuts = {
    'D_MIGR_IN_KM2': np.array(cutp).tolist()}
cuts

#making a dictionary out of cut values named cutp

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
#error 15 fix. Mac only.

In [ ]:
bins = sc.woebin(dfclean_nogeo.loc[:,["internet_access_grand_total", "target_label"]], 
                 y = "target_label", breaks_list = cuts, positive = 0)

sc.woebin_plot(bins) 
print(bins)

In [ ]:
dfclean_nogeo.loc[:,["internet_access_grand_total", "target_label"]]